In [1]:
import pandas as pd
from pathlib import Path
import sys
import re

In [2]:
def read_text(path: Path) -> str:
    # Read as UTF-8; replace undecodable bytes so the comparison can proceed
    return path.read_text(encoding="utf-8", errors="replace")

In [3]:
def strip_comments_and_whitespace(text: str) -> str:
    """
    Text-only minifier:
      1) For each line, drop everything from the first '#' to end-of-line.
      2) Remove ALL whitespace (spaces, tabs, newlines, etc.).
    NOTE: This treats '#' inside strings as comment starts too (by request: treat as text).
    """
    lines = []
    for line in text.splitlines():
        # Remove from first '#' onward
        hash_idx = line.find('#')
        if hash_idx != -1:
            line = line[:hash_idx]
        lines.append(line)
    no_comments = "".join(lines)
    # Drop all whitespace characters
    return re.sub(r"\s+", "", no_comments)

In [4]:
def edit_distance(a: str, b: str) -> int:
    """
    Levenshtein distance (insert/delete/substitute cost = 1).
    Space-optimized to O(min(n,m)) memory.
    """
    if a == b:
        return 0
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n

    prev = list(range(n + 1))
    curr = [0] * (n + 1)

    for j in range(1, m + 1):
        curr[0] = j
        bj = b[j - 1]
        for i in range(1, n + 1):
            cost = 0 if a[i - 1] == bj else 1
            curr[i] = min(
                prev[i] + 1,      # deletion
                curr[i - 1] + 1,  # insertion
                prev[i - 1] + cost,  # substitution
            )
        prev, curr = curr, prev
    return prev[n]


In [5]:


def read_csv_file(file_name: str) -> pd.DataFrame:
    if "__file__" in globals():
        base_dir = Path(__file__).resolve().parent
    else:
        base_dir = Path.cwd()

    csv_path = base_dir / file_name

    print(f"📄 CSV path: {csv_path}")
    print(f"🔍 Trying to read: {csv_path.resolve()}")

    if not csv_path.exists():
        raise FileNotFoundError(f"❌ CSV file not found: {csv_path}")

    try:
        df = pd.read_csv(csv_path)
        print(f"✅ Loaded CSV with {len(df)} rows × {len(df.columns)} columns.")
    except Exception as e:
        raise RuntimeError(f"Failed to read CSV: {e}")

    return df


In [6]:
df_decompiled_syntax_errors = read_csv_file('../dataset/decompiled_syntax_errors.csv')

📄 CSV path: /home/diogenes/pylingual_colaboration/pylingual_download/code/utils/../dataset/decompiled_syntax_errors.csv
🔍 Trying to read: /home/diogenes/pylingual_colaboration/pylingual_download/code/dataset/decompiled_syntax_errors.csv
✅ Loaded CSV with 383 rows × 5 columns.


In [7]:
import pandas as pd
from pathlib import Path

# Storage for results
results = []

print("=" * 80)
print("🧩 Starting decompiled file comparison")
print("=" * 80)

total_rows = len(df_decompiled_syntax_errors)
print(f"Total records to process: {total_rows}\n")
df_previous_results = pd.read_csv("edit_distance_results.csv")

for idx, row in df_decompiled_syntax_errors.iterrows():
    file_hash = row["file_hash"]
    file_name = row["file"]
    raw_name = file_name.split(".")[0].replace("decompiled_", "")

    if file_hash in df_previous_results["file_hash"].values:
        continue

    # Paths
    path1 = Path(f"../results/experiment_outputs/20251110T012915Z/daf95c71075048e1b3458c3c109344fd/{file_hash}/syntax_repaired_{file_name}")
    path2 = Path(f"/home/diogenes/pylingual_colaboration/pypi_downloaded/{file_hash}/{file_name}")
    path3 = Path(f"/home/diogenes/pylingual_colaboration/pypi_downloaded/{file_hash}/{raw_name}.py")

    print("-" * 80)
    print(f"[{idx+1}/{total_rows}] 🔍 Comparing files for hash: {file_hash}")
    print(f"  📄 File 1   : {path1.name}")
    print(f"  📄 File 2   : {path2.name}")
    print(f"  📄 File 3   : {path3.name}")

    # Default values in case of missing files
    d32 = None
    d31 = None
    status = "OK"

    # Check all files exist
    missing = [str(fp) for fp in (path1, path2, path3) if not fp.is_file()]
    if missing:
        print("❌ Missing files:")
        for m in missing:
            print(f"   - {m}")
        status = "missing"
    else:
        try:
            s1 = strip_comments_and_whitespace(read_text(path1))
            s2 = strip_comments_and_whitespace(read_text(path2))
            s3 = strip_comments_and_whitespace(read_text(path3))

            d32 = edit_distance(s3, s2)
            d31 = edit_distance(s3, s1)

            print(f"✅ Edit Distance (lookup vs decompiled)  : {d32}")
            print(f"✅ Edit Distance (lookup vs repaired)    : {d31}")
        except Exception as e:
            print(f"⚠️ Error during processing: {e}")
            status = "error"

    results.append({
        "file_hash": file_hash,
        "decompiled_file_name": file_name,
        "raw_file_name": path3.name,
        "d_lookup_vs_decompiled": d32,
        "d_lookup_vs_repaired": d31,
    })

    print("-" * 80)

    # Uncomment to test only one iteration
    # break

# --- Convert results to DataFrame ---
results_df = pd.DataFrame(results)
print("\n" + "=" * 80)
print("🏁 Finished processing all decompiled file comparisons.")
print("=" * 80)

print("\n📊 Results summary:")
print(results_df.head())




🧩 Starting decompiled file comparison
Total records to process: 383

--------------------------------------------------------------------------------
[10/383] 🔍 Comparing files for hash: 01b65e4670e382c820da8140c5b459602c122feff7c1bc9e9809776085a78e46521655f3f0f34f502f8062a71910609ac7c3bb839eca4a4adc5b240d544862e0
  📄 File 1   : syntax_repaired_decompiled_fmgr_firewall_address6template.cpython-312.py
  📄 File 2   : decompiled_fmgr_firewall_address6template.cpython-312.py
  📄 File 3   : fmgr_firewall_address6template.py
❌ Missing files:
   - ../results/experiment_outputs/20251110T012915Z/daf95c71075048e1b3458c3c109344fd/01b65e4670e382c820da8140c5b459602c122feff7c1bc9e9809776085a78e46521655f3f0f34f502f8062a71910609ac7c3bb839eca4a4adc5b240d544862e0/syntax_repaired_decompiled_fmgr_firewall_address6template.cpython-312.py
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
[16/383] 

In [8]:
# --- Optional: Save results ---
output_csv = Path("../dataset/decompiled_comparison_results.csv")
results_df.to_csv(output_csv, index=False)
print(f"\n💾 Results saved to: {output_csv.resolve()}")


💾 Results saved to: /home/diogenes/pylingual_colaboration/pylingual_download/code/dataset/decompiled_comparison_results.csv
